# Demonstration of anticipated end-to-end creation of Orbit PreCalculations
 - This is the expected usage to go from a newly-fitted orbit, to the storage of cheby polynomials & nightly-healpix in a database
 
 - At present (20200618) this is incomplete and intended as a schematic to aid development

#### Basic imports ...

In [ ]:
import time
import numpy as np
import scipy.stats as stats
import math
import random
from collections import defaultdict
import os
import sys
from collections import Counter
import glob 
import warnings
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import importlib
from astropy.time import Time
import pickle


# -------------------------------------------------------------------------------------
# Local imports
# -------------------------------------------------------------------------------------

# parent directory is */cheby_checker
HEAD_DIR = os.path.dirname(os.path.realpath(os.getcwd())) 
sys.path.append(os.path.join(HEAD_DIR))
print(f' HEAD_DIR: {HEAD_DIR} ')

# directory with sample data for development
DATA_DIR = os.path.join(HEAD_DIR, 'dev_data')
print(f' DATA_DIR: {DATA_DIR} ')

# import nbody-related code from main cheby_checker directory
from cheby_checker import mpc_nbody, parse_input, orbit_cheby, precalc, sql
importlib.reload(mpc_nbody)
importlib.reload(precalc)

"""
# orbit_cheby & nbody_reader imports 
#sys.path.append( "/Users/matthewjohnpayne/Envs/orbit_cheby/orbit_cheby/" )
from orbit_cheby import orbit_cheby
from orbit_cheby import nbody_reader
importlib.reload(orbit_cheby)
importlib.reload(nbody_reader)

# obs_pos import(s) 
from orbit_cheby import obs_pos
importlib.reload(obs_pos)

# sql import(s) 
from orbit_cheby import sql
importlib.reload(sql)

# PreCalc import(s) 
from orbit_cheby import precalc
importlib.reload(precalc)
"""

### (1) Use *mpc_nbody*
 - Perform NBody simulation on results from OrbFit
 - mpc_nbody is a convenience wrapper around Holman's version of reboundx
 - The notebook *Demonstate_Functionality_mpc_nbody.ipynb* contains more detailed demonstrations

In [ ]:
%%time
importlib.reload(mpc_nbody)

# Define some files that have data in them 
filenames = [os.path.join(DATA_DIR, file)
              for file in ['30101.eq0_horizons', '30102.eq0_horizons']]

# First, let's initiate the class with an input file:
Sim = mpc_nbody.NbodySim(filenames[0], 'eq')

# Now run the integrator, by calling the object. 
Sim(tstep=20, trange=600)

#The results are all available inside the object now:
for item in Sim.__dict__ : print(f'item:type = {item, type(Sim.__dict__[item])} ')

### (2) Use *orbit_cheby*
 - Load arrays of simulation data into Multi-Sector-Cheby object(s)

In [ ]:
# Use the MSC_Loader to do all of the work to declare and populate a list of MSC objects
#
MSCs = orbit_cheby.MSC_Loader(FROM_ARRAY = True , 
                                primary_unpacked_provisional_designations = filenames[0], 
                                times_TDB = Sim.output_times, 
                                statearray = Sim.output_vectors).MSCs

### (3) Do pre-calcs and store in db *orbit_cheby*
 - Uses the chebyshev coeffs in the MSCs to calculate nightly healpix
 - Stores the chebyshev coeffs & the nightly healpix in a database

In [ ]:
# Declare a "PreCalc" object 
P = precalc.PreCalc()

# I'mnot sure why P.upsert requires an observatory position.
# The MSC are independent of the observer. 
# The observer is only relevant once we want to query for "where is object at date-time?"

# To make this work, I'll just fudge a fake number for now:
# Ugh, why does this think the xyz needs to be shape (3, 20000)?
observatoryXYZ = np.zeros([3, 20000])

# Do the upsert 
P.upsert( MSCs , observatoryXYZ)